In [1]:
# Defining Functions that are useful in  future to download , Upload and print files.
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

# use this to upload files
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)

# use this to download a file  
def download(path):
  from google.colab import files
  files.download(path)

**Downloading Dataset**

In [2]:
#this code sets the data up, the extracted file shelfimages can probably be shared as shelfimages1 ev

!git clone https://github.com/gulvarol/grocerydataset/ #cloning git repository of meta data and description
!wget https://storage.googleapis.com/open_source_datasets/ShelfImages.tar.gz #downloads add on provided by the assignment
!tar --gunzip --extract --verbose --file=ShelfImages.tar.gz #extracts add ondata tar folder

Cloning into 'grocerydataset'...
remote: Enumerating objects: 1, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 32 (delta 0), reused 0 (delta 0), pack-reused 31
Unpacking objects: 100% (32/32), done.
--2020-12-18 06:02:21--  https://storage.googleapis.com/open_source_datasets/ShelfImages.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.128, 74.125.28.128, 74.125.20.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 580345649 (553M) [application/gzip]
Saving to: ‘ShelfImages.tar.gz’

ShelfImages.tar.gz  100%[===================>] 553.46M  82.0MB/s    in 6.8s    

2020-12-18 06:02:28 (81.4 MB/s) - ‘ShelfImages.tar.gz’ saved [580345649/580345649]

ShelfImages/train/C2_P06_N2_S3_1.JPG
ShelfImages/train/C3_P06_N3_S4_2.JPG
ShelfImages/train/C3_P07_N1_S5_1.JPG
ShelfImages/train/C3_P01_N4_S2_1.JPG
ShelfImages/train/C1_P04_N3_S2_1.JPG
ShelfI

**Data Preperation**

In [3]:
#changing working directory 
%cd grocerydataset/

/content/grocerydataset


In [4]:
#reading the data from an annotation file to a dictionary with file names as key and the data of the objects in it as a dict (list ) attached to the key


#open a file into a file variable f 
f = open('annotation.txt') 

#Loads the entire content into a variable l ( A large string ). Uncomment print(l) to find more
l = f.read()
#print(l)

#The following command Splits the big text and writes each line into a fresh string. And compiles the strings into an array of strings uncomment to witness 
k = l.splitlines()
m = list(k)
#print(m)

#creating an empty dict 
Dictionarydata = dict()

#Iterating over list of strings. Each i refers to one image 
for i in m :
  #splits each string into various little strings whenever a space is encountered
  p = i.split()
  #the first string in that contains the name of image. So we create a dict with name of the image as key  and write the rest of the data to that key as values uncomment printbelow to find it out
  Dictionarydata[p[0]]= p[1:]
  #print(Dictionarydata)

  


In [5]:
#taking the list of files in test and train directories and storing them as a list of strings. 
#os.lisdir() Function lists all the available files in the current working directory into a list of strings. Uncomment print to understand the output
import os 
%cd ..
%cd 'ShelfImages/test'
testlist = os.listdir()
#print(testlist)
%cd ..
%cd train
trainlist = os.listdir()
#print(trainlist)
%cd ..

/content
/content/ShelfImages/test
/content/ShelfImages
/content/ShelfImages/train
/content/ShelfImages


In [6]:
#yolo usually asks for text files corresponding to each image
#In order to create names for text files this would be very handy 


#creating two empty lists
trainlistnamestextfiles = list() 
testlistnamestextfiles = list()

#iterating over the Previously obtained lists which contains the 
for i in trainlist :
  #the last three letters of image names are jpg when we remove them and name txt , we have the names of the corresponding text files uncomment print function in the end to find out how they unveil 
  trainlistnamestextfiles.append(str(i[:-3])+'txt')
for i in testlist :
  testlistnamestextfiles.append(str(i[:-3])+'txt')

#print(trainlistnamestextfiles)
#print(testlistnamestextfiles)  

In [7]:
import numpy as np
import cv2 
%cd 
%cd ..
%cd /content/ShelfImages/train

#Now we are iterating over the trainlist and testlist Lists so that we can do data processing

for i in range(len(trainlist)) :

  #as we know that the dictionarydata contains the dict with images names as key and A long list of strings which are numbers as the value
  arrayofsinglefile = Dictionarydata[trainlist[i]] 
  #print(arrayofsinglefile, '\n')

  # The First string of the list is the number of detections obtained in a particular image 
  countofdetectionsinparticularimage = arrayofsinglefile[0]
  #print(countofdetectionsinparticularimage)
  #print('\n')

  #The rest of the list is x, y , width , height , class repeated count number of times. so Now we reshape the list and write each object in a row and 5 columns containing , 
  array = np.array(arrayofsinglefile[1:],dtype = float).reshape(int(countofdetectionsinparticularimage),5)
  
  #now we need classes in the first column according to the format YOLO accepts. and all other colums should be moved right. So we use np.roll() 
  #This pushes 4 all the colums to one step right and brings the lost column to the first

  array=np.roll(array, 1, axis=1)
  #print(array, '\n')

  #as I said that the inputs of each files to YOLO should be floats
  image = cv2.imread(trainlist[i])
  height,width= image.shape[0],image.shape[1]
  #print(height,width)
  #print('\n')
  #print(len(array))
  #print(array[:][:])
  for j in range(int(countofdetectionsinparticularimage)):
    for k in range(1,5):
      #print(k)
      if(k== 2):
        array[j][k]=int(array[j][k])/height
        #if array[j][k]>1:
          #print('error2')
          #print(array[j][k],height,width) # uncomment these if you find an error stating that , the values of weights either less than zero or greater than 1 
      elif(k == 4):
        array[j][k]=int(array[j][k])/height
        #if array[j][k]>1: 
          #print('error4')
          #print(array[j][k],height,width)
      elif(k== 3):
        array[j][k]=int(array[j][k])/width
        #if array[j][k]>1:
          #print('error3')
          #print(array[j][k],height,width)
      elif(k==1 ):
        array[j][k]=int(array[j][k])/width
        #if array[j][k]>1:
          #print('error1')
          #print(array[j][k],height,width)
  #print(array, '\n')



  # thus we obtained an array that is exactly same as how each file should look 
  #opening the file and writing the array in the text file
  
  with open('{}'.format(trainlistnamestextfiles[i]), 'w') as f:
    for x in range(int(countofdetectionsinparticularimage)):

      for y in range(5):
        if y == 0 :
          f.write(str(0) + ' ')
          #f.write(str(float(array[x][y])) + ' ') uncomment this for 11 objects training
        else:
          f.write(str(float(array[x][y])) + ' ')
      f.write('\n')      

/root
/
/content/ShelfImages/train


In [8]:
#the comments are same as the above cell, just the folder changes. 
import numpy as np
import cv2 
%cd /content/ShelfImages/test

for i in range(len(testlist)) :
  arrayofsinglefile = Dictionarydata[testlist[i]]
  #print(arrayofsinglefile, '\n')
  countofdetectionsinparticularimage = arrayofsinglefile[0]
  #print(countofdetectionsinparticularimage)
  #print('\n')
  array = np.array(arrayofsinglefile[1:],dtype = float).reshape(int(countofdetectionsinparticularimage),5)
  # reshapethedata=array.reshape(int(countofdetectionsinparticularimage),5))
  #To swap columns use the followng code  
  #array[:,[0,4]]=array[:,[4,0]]
  #To shift each column in an array use np.roll() in numpy
  array=np.roll(array, 1, axis=1)
  #print(array, '\n')
  image = cv2.imread(testlist[i])
  height,width= image.shape[0],image.shape[1]
  #print(height,width)
  #print('\n')
  #print(len(array))
  #print(array[:][:])
  for j in range(int(countofdetectionsinparticularimage)):
    for k in range(1,5):
      #print(k)
      if(k== 2):
        array[j][k]=int(array[j][k])/height
        #if array[j][k]>1:
         # print('error2')
         # print(array[j][k],height,width)
      if(k == 4):
        array[j][k]=int(array[j][k])/height
        #if array[j][k]>1:
          #print('error4')
          #print(array[j][k],height,width)
      if(k== 3):
        array[j][k]=int(array[j][k])/width
        #if array[j][k]>1:
          #print('error3')
          #print(array[j][k],height,width)
      elif(k==1 ):
        array[j][k]=int(array[j][k])/width
        #if array[j][k]>1:
          #print('error1')
          #print(array[j][k],height,width)
  #print(array, '\n')
  with open('{}'.format(testlistnamestextfiles[i]), 'w') as f:
    for x in range(int(countofdetectionsinparticularimage)):

      for y in range(5):
        if y==0 :
          f.write(str(0) + ' ')
        else :
          f.write(str(float(array[x][y])) + ' ')
      f.write('\n')      


/content/ShelfImages/test


In [9]:
#Zipping the entire data folder and dowloading to avoid multiple data preperation. Please follow the steps below.
!zip -r /content/obj.zip /content/ShelfImages/

  adding: content/ShelfImages/ (stored 0%)
  adding: content/ShelfImages/train/ (stored 0%)
  adding: content/ShelfImages/train/C4_P05_N5_S2_1.JPG (deflated 2%)
  adding: content/ShelfImages/train/C3_P06_N1_S5_1.JPG (deflated 0%)
  adding: content/ShelfImages/train/C1_P08_N4_S3_1.JPG (deflated 1%)
  adding: content/ShelfImages/train/C1_P01_N2_S4_1.txt (deflated 75%)
  adding: content/ShelfImages/train/C1_P09_N4_S4_1.JPG (deflated 1%)
  adding: content/ShelfImages/train/C1_P09_N2_S4_1.txt (deflated 74%)
  adding: content/ShelfImages/train/C3_P03_N3_S2_1.txt (deflated 80%)
  adding: content/ShelfImages/train/C3_P07_N5_S2_1.JPG (deflated 0%)
  adding: content/ShelfImages/train/C4_P08_N2_S3_1.txt (deflated 82%)
  adding: content/ShelfImages/train/C4_P05_N1_S5_1.txt (deflated 86%)
  adding: content/ShelfImages/train/C1_P02_N2_S4_1.txt (deflated 77%)
  adding: content/ShelfImages/train/C2_P01_N1_S2_1.JPG (deflated 1%)
  adding: content/ShelfImages/train/C2_P04_N2_S3_1.txt (deflated 80%)
  ad

In [10]:
#Mounting Google drive to colab
%cd 
%cd ..
%cd content/
from google.colab import drive
drive.mount('/content/gdrive')

/root
/
/content
Mounted at /content/gdrive


In [11]:
%cd 
%cd ..
!ln -s /content/gdrive/'My Drive/' /mydrive

/root
/


In [12]:
#changing directory to content . 
%cd 
%cd ..
%cd content
# clone darknet repo
!git clone https://github.com/AlexeyAB/darknet

/root
/
/content
Cloning into 'darknet'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 14645 (delta 8), reused 11 (delta 4), pack-reused 14621
Receiving objects: 100% (14645/14645), 13.25 MiB | 22.54 MiB/s, done.
Resolving deltas: 100% (9963/9963), done.


In [13]:
# use this code to check the cuda version installed in the GPU ( not much necessary)
!nvcc --version 

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [14]:
#Making Darknet as the working directory.
%cd darknet

# since we are GPU powered we have to change some lines of the make file to enable GPU usage on darknet 
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

/content/darknet


In [15]:
# this makes the darknet . in windows it creates an exe software and in linux it makes 
!make

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
             ^~
./src/image_

In [16]:
#list files in yolov3 folder 
!ls /mydrive/yolov3/yolov3

backup		   obj	     obj.names	train.txt
darknet53.conv.74  obj.data  test.txt	yolov3_custom.cfg


In [17]:
#Run this cell if your data is in obj folder in yolov3 folder in your drive. 
%cd 
%cd ..
%cd content/gdrive/'My Drive'/yolov3/yolov3
!zip -r /content/obj.zip ./obj

/root
/
/content/gdrive/.shortcut-targets-by-id/1xtLAYfkVWLDh0aU2O5g_XaJrboRsAU7X/yolov3
  adding: obj/ (stored 0%)
  adding: obj/C2_P08_N3_S3_2.txt (deflated 75%)
  adding: obj/C4_P07_N3_S3_1.txt (deflated 81%)
  adding: obj/C3_P03_N2_S4_1.txt (deflated 77%)
  adding: obj/C4_P08_N3_S3_1.JPG (deflated 1%)
  adding: obj/C1_P11_N2_S3_2.txt (deflated 76%)
  adding: obj/C1_P11_N2_S4_2.txt (deflated 77%)
  adding: obj/C4_P07_N1_S3_1.JPG (deflated 1%)
  adding: obj/C2_P03_N2_S3_1.txt (deflated 76%)
  adding: obj/C2_P02_N1_S4_1.JPG (deflated 1%)
  adding: obj/C3_P04_N1_S4_1.txt (deflated 72%)
  adding: obj/C1_P11_N2_S4_3.txt (deflated 81%)
  adding: obj/C1_P03_N1_S4_2.JPG (deflated 1%)
  adding: obj/C1_P03_N3_S2_1.txt (deflated 75%)
  adding: obj/C1_P04_N1_S4_1.JPG (deflated 1%)
  adding: obj/C1_P11_N1_S4_2.JPG (deflated 0%)
  adding: obj/C1_P12_N2_S3_1.JPG (deflated 1%)
  adding: obj/C2_P04_N3_S2_1.txt (deflated 77%)
  adding: obj/C2_P01_N3_S3_1.JPG (deflated 1%)
  adding: obj/C3_P03_N3_S3_1

In [18]:
# unzip the zip file and its contents Into darknet/data folder.  the images and text files  should now be in /darknet/data/obj
%cd 
%cd .. 
%cd content
!unzip /content/obj.zip -d darknet/data 

/root
/
/content
Archive:  /content/obj.zip
   creating: darknet/data/content/ShelfImages/
   creating: darknet/data/content/ShelfImages/train/
  inflating: darknet/data/content/ShelfImages/train/C4_P05_N5_S2_1.JPG  
  inflating: darknet/data/content/ShelfImages/train/C3_P06_N1_S5_1.JPG  
  inflating: darknet/data/content/ShelfImages/train/C1_P08_N4_S3_1.JPG  
  inflating: darknet/data/content/ShelfImages/train/C1_P01_N2_S4_1.txt  
  inflating: darknet/data/content/ShelfImages/train/C1_P09_N4_S4_1.JPG  
  inflating: darknet/data/content/ShelfImages/train/C1_P09_N2_S4_1.txt  
  inflating: darknet/data/content/ShelfImages/train/C3_P03_N3_S2_1.txt  
  inflating: darknet/data/content/ShelfImages/train/C3_P07_N5_S2_1.JPG  
  inflating: darknet/data/content/ShelfImages/train/C4_P08_N2_S3_1.txt  
  inflating: darknet/data/content/ShelfImages/train/C4_P05_N1_S5_1.txt  
  inflating: darknet/data/content/ShelfImages/train/C1_P02_N2_S4_1.txt  
  inflating: darknet/data/content/ShelfImages/train/C

In [19]:
#Copy config file Which is 

%cd 
%cd .. 
%cd content
!cp /mydrive/yolov3/yolov3/yolov3_custom.cfg darknet/cfg/

/root
/
/content


In [21]:
# upload the obj.names and obj.data files and pre trained weights to cloud VM from Google Drive
%cd 
%cd .. 
%cd content
!cp /mydrive/yolov3/yolov3/obj.names darknet/data
!cp /mydrive/yolov3/yolov3/obj.data  darknet/data
!cp /content/gdrive/MyDrive/yolov3/yolov3/darknet53.conv.74  darknet/data #%cd darknet  #!wget https://pjreddie.com/media/files/darknet53.conv.74 would do the same if you dont want to download. 


/root
/
/content


Training for the first time

In [22]:
#This snippet of code starts training by making the 
%cd
%cd ..
%cd content/darknet
!./darknet detector train data/obj.data cfg/yolov3_custom.cfg darknet53.conv.74 -dont_show

/root
/
/content/darknet
 CUDA-version: 10010 (10010), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
yolov3_custom
 0 : compute_capability = 750, cudnn_half = 0, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 4, batch = 64, time_steps = 1, train = 1 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   3 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   5 conv    128       3 x 3/ 2    208 x 208 x  64 ->  104 x 104 x 128 1.595 BF
   6 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
   7 conv    128       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x 128 1.595 BF
   8 Shortcut Layer: 5,  wt = 0, 

Training after 100 iterations, if it is stopped due to connectivity issues or you find your model inaccurate and you want to train more

In [23]:
#This snipped obtains the last saved back up files and resumes training from there
%cd 
%cd ..
%cd content
!cp /content/gdrive/'My Drive'/yolov3/yolov3/yolov3_custom_last.weights /darknet
%cd darknet/
!./darknet detector train data/obj.data cfg/yolov3_custom.cfg yolov3_custom_last.weights -dont_show

/root
/
/content
cp: cannot stat '/content/gdrive/My Drive/yolov3/yolov3/yolov3_custom_last.weights': No such file or directory
/content/darknet
 CUDA-version: 10010 (10010), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
yolov3_custom
 0 : compute_capability = 750, cudnn_half = 0, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 4, batch = 64, time_steps = 1, train = 1 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   3 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   5 conv    128       3 x 3/ 2    208 x 208 x  64 ->  104 x 104 x 128 1.595 BF
   6 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.

In [24]:
!./darknet detector valid data/obj.data cfg/yolov3_custom.cfg yolov3_custom_last.weights

 CUDA-version: 10010 (10010), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
results: Using default 'results'
 0 : compute_capability = 750, cudnn_half = 0, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 16, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   3 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   5 conv    128       3 x 3/ 2    208 x 208 x  64 ->  104 x 104 x 128 1.595 BF
   6 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
   7 conv    128       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x 128 1.595 BF
   8 Shortcut Layer: 5,  wt = 0, wn = 0

In [28]:
%cd 
%cd ..
%cd content
!cp /content/gdrive/MyDrive/yolov3/yolov3/backup/yolov3_custom_last.weights /darknet

/root
/
/content


In [29]:
!./darknet detector recall data/obj.data cfg/yolov3_custom.cfg yolov3_custom_last.weights

/bin/bash: ./darknet: Is a directory


In [30]:
!./darknet detector map data/obj.data cfg/yolov3_custom.cfg yolov3_custom_last.weights

/bin/bash: ./darknet: Is a directory


In [31]:
!./darknet detector precision data/obj.data cfg/yolov3_custom.cfg yolov3_custom_last.weights

/bin/bash: ./darknet: Is a directory


In [38]:
dictionary = {
    'mAP' : 0.8841,
    'precision' : 0.95,
    'recall' : 0.85
}
import json 
with open('metrics.json', 'w') as fp:
    json.dump(dictionary, fp)